# Make corn_vmd_weekly.csv

In [4]:
#VMD instalation
!pip install vmdpy

# Imports
import pandas as pd
import numpy as np
from vmdpy import VMD

# Pull corn data
data_url = 'https://raw.githubusercontent.com/ptraver/data/main/corn_daily_V2.csv'
corn_df = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')

# fix bad name
corn_df.columns = ['value']

# take only 1974 - 2017
corn_df = corn_df['1974-01-01':'2017-12-31']

In [5]:
# downsample to weekly
corn_df = corn_df.resample('W').mean()

In [6]:
# Make VMD

#. some sample parameters for VMD  
alpha = 2000       # moderate bandwidth constraint  
tau = 0.           # noise-tolerance (no strict fidelity enforcement)  
K = 7              # 7 modes -- same amount as emd on this set
DC = 0             # no DC part imposed  
init = 0           # initialize omegas uniformly  
tol = 1e-3

u = VMD(corn_df['value'].values, alpha, tau, K, DC, init, tol)

# sum up all modes except highest frequency
v = np.zeros((len(u[0][0])))
for i in range(K - 1):
  v += u[0][i]

# Make a corn_vmd dataframe
corn_vmd = pd.DataFrame(v, columns=['value'], index=corn_df.index[:-1])

In [7]:
# Save dataframe
corn_vmd.to_csv('corn_vmd_weekly.csv')

# End